In [1]:
#лабораторная_работа_1

In [1]:
#1 черное, белое, красное изображения и градиент
import random
from PIL import Image
import numpy as np
import math

h = 200  #размерность
w = 200
#черное
paint1 = np.zeros((h, w), dtype=np.uint8)
image1 = Image.fromarray(paint1, mode='L')  
image1.save('black.jpg')      
#белое
paint2 = 255 * np.ones((h, w), dtype=np.uint8)
image2 = Image.fromarray(paint2, mode='L')
image2.save('white.jpg')
#красное
paint3 = np.full((h, w, 3), (255, 0, 0), dtype=np.uint8)
image3 = Image.fromarray(paint3, mode='RGB')
image3.save('red.jpg')
#градиент
paint4 = np.zeros((h, w, 3), dtype=np.uint8)
for i in range(h):
    for j in range(w):
        paint4[i, j, :] = [(i + j) % 256]*3
image4 = Image.fromarray(paint4, mode='RGB')
image4.save('gradient.jpg')

In [2]:
#2 отрисовка прямых линий

#1вариант
image = np.zeros((200, 200), dtype=np.uint8)
x0, y0 = 100, 100
alpha = 2 * math.pi / 13
for i in range(13):
    x1 = 100 + 95 * math.cos(alpha * i)
    y1 = 100 + 95 * math.sin(alpha * i)
    for t in range(101):
        t /= 100
        x = int(x0 * (1 - t) + x1 * t)
        y = int(y0 * (1 - t) + y1 * t)
        image[y, x] = 255

image = Image.fromarray(image, mode='L')
image.save('FirstStar.png')

#2вариант
image = np.zeros((200, 200), dtype=np.uint8)
x0, y0 = 100, 100
alpha = 2 * math.pi / 13
for i in range(13):
    x1 = int(100 + 95 * math.cos(alpha * i))
    y1 = int(100 + 95 * math.sin(alpha * i))
    for x in range(x0,x1):
        t = (x-x0) / (x1-x0)
        y = int(y1 * t + y0 * (1 - t))
        image[y, x] = 255
        

image = Image.fromarray(image, mode='L')
image.save("SecondStar.jpg")

#3вариант
image = np.zeros((200, 200), dtype=np.uint8)
alpha = 2 * math.pi / 13
for i in range(13):
    x0, y0 = 100, 100
    x1 = int(100 + 95 * math.cos(alpha * i))
    y1 = int(100 + 95 * math.sin(alpha * i))
    steep = False
    
    if(abs(x0 - x1) < abs(y0 - y1)):
        x0, y0 = y0, x0
        x1, y1 = y1, x1
        steep = True

    if(x0 > x1):
        x0, x1 = x1, x0
        y0, y1 = y1, y0

    for x in range(x0, x1):
        if(steep):
            t = (x - x0)/(x1 - x0)
            y = int(y0 * (1 - t) + y1 * t)   
            image[x, y]=255
            image[y, x]=255
        
image = Image.fromarray(image, mode='L')
image.save("ThirdStar.jpg")

#4вариант_Брезенхем
image = np.zeros((200, 200), dtype=np.uint8)
for i in range(13):
    x0, y0 = 100, 100
    x1 = int(100 + 95 * math.cos(alpha * i))
    y1 = int(100 + 95 * math.sin(alpha * i))
    steep = False
    
    if(abs(x0 - x1) < abs(y0 - y1)):
        x0, y0 = y0, x0
        x1, y1 = y1, x1
        steep = True
        
    if(x0 > x1):
        x0, x1 = x1, x0
        y0, y1 = y1, y0

    dx = x1 - x0
    dy = y1 - y0
    derror = abs(dy / dx)
    error = 0
    y = y0
    for x in range(x0, x1):
        if (steep):
            image[y, x] = 255
        else:
            image[x, y] = 255

        error += derror
        if (error > 0.5):
            if (y1 > y0):
                y += 1
            else:
                y -= 1
            error -= 1

image = Image.fromarray(image, mode='L')
image.save("FourthStar.jpg")

In [28]:
#3 Работа с трёхмерной моделью (вершины)
class Vert:
    def __init__(self, x, y, z):               #храним информацию о точке в трехмерном пространстве
        self.x = x
        self.y = y
        self.z = z

class Read:
    def __init__(self, filename):
        self.array = []
        self.normals = []
        try:
            with open(filename, 'r') as f:
                lines = f.readlines()[4:] #читаем файл с 4ой строки
                for line in lines:
                    points = line.split()
                    if (points[0] == 'v'): #если строка начинается с v, добавляем в список array
                        self.array.append(Vert(float(points[1]), float(points[2]), float(points[3])))
                    if (points[0] == 'vn'): #если строка начинается с vn, добавляем в список normals
                        self.normals.append(Vert(float(points[1]), float(points[2]), float(points[3]))) 
        except FileNotFoundError:   #если ошибки при чтении файла и пр.
            print(f"File {filename} not found")   
        except Exception as e:
            print(f"Error: {e}")

data = Read('fox.obj') #сохраняем в data

In [4]:
#4 Отрисовка вершин трёхмерной модели
from PIL import Image, ImageDraw

vertexes = np.zeros((1000, 1000), dtype=np.uint8)
vertexes = Image.fromarray(vertexes)

for point in data.array:                       #проходимся по точкам в array(data), применяем масштабирование и смещение
    x = int(float(point.x)) * 7 + 500
    y = -int(float(point.y)) * 7 - 150
    vertexes.putpixel((x, y), 255)            #присвоение точкам белого цвета(255)

vertexes.save('vertexes.jpg')

In [5]:
#5 Работа с трёхмерной моделью (полигоны)
polygons = []
with open('fox.obj', 'r') as f:                    #открываем файл
    for line in f:
        if line.startswith('f'):                   #если строка начинается с f
            vertices = line.strip().split()[1:]    #удаляем пробелы; разделяем на элементы
            polygon = []
            for vertex in vertices:
                polygon.append(str(vertex.split('/')[0])) #берём части,отделенные '/'; сохраняем в список
            polygons.append(tuple(polygon))  #по окончанию цикла, преобр. список в кортеж и сохраняем в список polygons
f.close()

#print(polygons[:10])


In [6]:
#6 Отрисовка рёбер трёхмерной модели
def drawing_edges(x0, y0, x1, y1, image): 
    x0, x1 = x0 * 7 + 500, x1 * 7 + 500
    y0, y1 = -y0 * 7 - 150, -y1 * 7 - 150                     #алг. Брезенхема
    steep = False  
    
    if (abs(x0 - x1) < abs(y0 - y1)):
        x0, y0 = y0, x0
        x1, y1 = y1, x1
        steep = True
        
    if (x0 > x1):
        x0, x1 = x1, x0
        y0, y1 = y1, y0
        
    dx = x1 - x0
    dy = y1 - y0
    derror = abs(dy / dx)
    error = 0
    y = y0
    for x in range(int(x0), int(x1)):
        if (steep):
            image.putpixel((int(y), int(x)), 255)
        else:
            image.putpixel((int(x), int(y)), 255)
        error += derror
        if (error > 0.5):
            if (y1 > y0):
                y += 1
            else:
                y -= 1
            error -= 1.0


for line in polygons:  #Проходим по многоуг-ам из polygons
    for i, v in enumerate(line): #Проходим по всем вершинам
        j = (i + 1) % len(line) #Получаем индекс следующей вершины
        #Получаем координаты текущей и следующей вершин
        (x0, y0), (x1, y1) = (data.array[int(v) - 1].x, data.array[int(v) - 1].y), (data.array[int(line[j]) - 1].x, data.array[int(line[j]) - 1].y)
        drawing_edges(x0, y0, x1, y1, vertexes) #вызываем ф. отрисовки
vertexes.save('drawing_edges.jpg')

In [7]:
#лабораторная_работа_2

In [8]:
#7 Барицентрические координаты
def barycentric(x, y, vrtxs, data):
    #присваиваем значения (вершины треугольника)
    x0, x1, x2 = [float(data.array[int(vrtxs[i]) - 1].x) * 7 - 500 for i in range(3)]
    y0, y1, y2 = [float(data.array[int(vrtxs[i]) - 1].y) * 7 + 150 for i in range(3)]
    #вычисляем по формуле
    lam0 = ((x1 - x2) * (y - y2) - (y1 - y2) * (x - x2)) / ((x1 - x2) * (y0 - y2) - (y1 - y2) * (x0 - x2))
    lam1 = ((x2 - x0) * (y - y0) - (y2 - y0) * (x - x0)) / ((x2 - x0) * (y1 - y0) - (y2 - y0) * (x1 - x0))
    lam2 = ((x0 - x1) * (y - y1) - (y0 - y1) * (x - x1)) / ((x0 - x1) * (y2 - y1) - (y0 - y1) * (x2 - x1))

    #if(lam0 + lam1 + lam2 != 1):
        #print("lam0+lam1+lam2!=1")
    return Vert(lam0, lam1, lam2) 

In [9]:
#8 Отрисовка треугольников

triangle = np.full((1000, 1000, 3), (0, 0, 0), dtype=np.uint8)

def drawing_triangle(vrtxs, data, matrix):
    x0, x1, x2 = [float(data.array[int(vrtxs[i]) - 1].x) * 7 - 500 for i in range(3)]
    y0, y1, y2 = [float(data.array[int(vrtxs[i]) - 1].y) * 7 + 150 for i in range(3)]
    #Определяем ограничивающий прямоугольник
    xmin, xmax = min(x0, x1, x2), max(x0, x1, x2)
    ymin, ymax = min(y0, y1, y2), max(y0, y1, y2)
    color = (random.uniform(0, 255), random.uniform(0, 255), random.uniform(0, 255)) #случайный цвет

    x_coords = np.arange(int(xmin), math.ceil(xmax))
    y_coords = np.arange(int(ymin), math.ceil(ymax))
    xx, yy = np.meshgrid(x_coords, y_coords, indexing='xy') #матрицы с коорд-ми по x и y

    coords = barycentric(xx, yy, vrtxs, data) 
    valid_coords = (coords.x >= 0) & (coords.y >= 0) & (coords.z >= 0) #проверяем координаты на положительность

    matrix[-yy[valid_coords], xx[valid_coords]] = color #рисуем пиксели с "валидными" координатами
                
#9 Тестирование функции
for i in range(0, 100, 30):
    drawing_triangle(polygons[i], data, triangle)
    triangle_test = Image.fromarray(triangle, mode='RGB')
    triangle_test.save('drawing_triangle_test.jpg')


In [10]:
#10 Отрисовка полигонов трёхмерной модели
for vrtxs in polygons:
    drawing_triangle(vrtxs, data, triangle)
    poly3d = Image.fromarray(triangle)
    poly3d.save('drawing_polygons.jpg') 

In [11]:
#лабораторная_работа_3

In [12]:
#11 Вычисление нормали к поверхности треугольника
def normal(vrtxs, data):
    #задаем координаты
    x0, x1, x2 = [float(data.array[int(vrtxs[i]) - 1].x) for i in range(3)]
    y0, y1, y2 = [float(data.array[int(vrtxs[i]) - 1].y) for i in range(3)]
    z0, z1, z2 = [float(data.array[int(vrtxs[i]) - 1].z) for i in range(3)]
    #вычисляем по формуле
    n_1 = [x1 - x0, y1 - y0, z1 - z0]
    n_2 = [x1 - x2, y1 - y2, z1 - z2]
    n = np.cross(n_1, n_2)
    
    return n

In [13]:
#12 Отсечение нелицевых граней
def norm_cos(vrtxs, data):
    light_direction = [0, 0, 1] #напр. света
    norm_vector = normal(vrtxs, data) #вычисление нормали
    #выч.косинус по формуле:
    cos = np.dot(norm_vector, light_direction) / (np.linalg.norm(norm_vector) * np.linalg.norm(light_direction))
    return cos   

def drawing_triangle_negative_cos(vrtxs, data, matrix): #измененная функция отрисовки с новым условием
    cos = norm_cos(vrtxs, data)
    if cos < 0: #меняем цикл отрисовки, рисуем полигоны с cos<0
        x0, x1, x2 = [float(data.array[int(vrtxs[i]) - 1].x) * 7 - 500 for i in range(3)]
        y0, y1, y2 = [float(data.array[int(vrtxs[i]) - 1].y) * 7 + 150 for i in range(3)]

        xmin, xmax = min(x0, x1, x2), max(x0, x1, x2)
        ymin, ymax = min(y0, y1, y2), max(y0, y1, y2)

        color = (random.uniform(0, 255), random.uniform(0, 255), random.uniform(0, 255))

        x_coords = np.arange(int(xmin), math.ceil(xmax))
        y_coords = np.arange(int(ymin), math.ceil(ymax))
        xx, yy = np.meshgrid(x_coords, y_coords, indexing='xy')

        coords = barycentric(xx, yy, vrtxs, data)
        valid_coords = (coords.x >= 0) & (coords.y >= 0) & (coords.z >= 0)

        matrix[-yy[valid_coords], xx[valid_coords]] = color
        
negcos = np.full((1000, 1000, 3), (0, 0, 0), dtype=np.uint8)  
#рисуем
for vrtxs in polygons:
    drawing_triangle_negative_cos(vrtxs, data, negcos)
    negativecos = Image.fromarray(negcos)
    negativecos.save('PolygonsWithNegativeCos.jpg')

In [14]:
#13 Базовое освещение
def drawing_triangle_basic_lighting(vrtxs, data, matrix):
    cos = norm_cos(vrtxs, data)
    if cos < 0:
        x0, x1, x2 = [float(data.array[int(vrtxs[i]) - 1].x) * 7 - 500 for i in range(3)]
        y0, y1, y2 = [float(data.array[int(vrtxs[i]) - 1].y) * 7 + 150 for i in range(3)]

        xmin, xmax = min(x0, x1, x2), max(x0, x1, x2)
        ymin, ymax = min(y0, y1, y2), max(y0, y1, y2)
    
        x_coords = np.arange(int(xmin), math.ceil(xmax))
        y_coords = np.arange(int(ymin), math.ceil(ymax))
        xx, yy = np.meshgrid(x_coords, y_coords, indexing='xy')  

        coords = barycentric(xx, yy, vrtxs, data)
        valid_coords = (coords.x >= 0) & (coords.y >= 0) & (coords.z >= 0)
        #теперь цвет не случайный, а пропорциональный косинусу
        matrix[-yy[valid_coords], xx[valid_coords]] = (-250 * cos, -100 * cos, -100 * cos)
        
bsclghtng = np.full((1000, 1000, 3), (0, 0, 0), dtype=np.uint8) 

for vrtxs in polygons:
    drawing_triangle_basic_lighting(vrtxs, data, bsclghtng)
    basiclighting = Image.fromarray(bsclghtng)
    basiclighting.save('basic_lighting.jpg')

In [16]:
#14  z-буфер
def drawing_zbuffer(vrtxs, data, matrix, zbuffer):
    cos = norm_cos(vrtxs, data)
    if cos < 0:        
        x0, x1, x2 = [float(data.array[int(vrtxs[i]) - 1].x) * 7 - 500 for i in range(3)]
        y0, y1, y2 = [float(data.array[int(vrtxs[i]) - 1].y) * 7 + 150 for i in range(3)]
        z0, z1, z2 = [float(data.array[int(vrtxs[i]) - 1].z) for i in range(3)]

        xmin, xmax = min(x0, x1, x2), max(x0, x1, x2)
        ymin, ymax = min(y0, y1, y2), max(y0, y1, y2)
        zmin, zmax = min(z0, z1, z2), max(z0, z1, z2)

        x_coords = np.arange(int(xmin), math.ceil(xmax))
        y_coords = np.arange(int(ymin), math.ceil(ymax))
        xx, yy = np.meshgrid(x_coords, y_coords, indexing='xy')

        for i in range(len(xx)):
            for j in range(len(xx[i])):
                coord = barycentric(xx[i][j], yy[i][j], vrtxs, data) #вычисляем барицентр. координаты
                if (coord.x >= 0) & (coord.y >= 0) & (coord.z >= 0): # если все положительные
                    zcoord = coord.x * z0 + coord.y * z1 + coord.z * z2 #вычисляем z-координату исходного полигона 
                    if zcoord < zbuffer[xx[i][j]][yy[i][j]]:#Если значение z-координаты меньше координаты z-буфера 
                        matrix[-yy[i][j]][xx[i][j]] = (-250 * cos, -100 * cos, -100 * cos) #рисуем
                        zbuffer[xx[i][j]][yy[i][j]] = zcoord #соответствующему элементу z-буфера присваиваем значение z.


z = np.full((1000, 1000, 3), (0, 0, 0), dtype=np.uint8)
zbuffer = np.inf * np.ones((1000, 1000), dtype=float) #буфер

for vrtxs in polygons:
    drawing_zbuffer(vrtxs, data, z, zbuffer)
    zbff = Image.fromarray(z, mode='RGB')
    zbff.save('zbuffer.jpg')

In [17]:
#лабораторная_работа_4

In [18]:
#15 Проективное преобразование
def new_barycentric(x, y, vrtxs, data, z_add): #новая функция для вычисления бар. координат
    x0, x1, x2 = [float(data.array[int(vrtxs[i]) - 1].x) for i in range(3)]
    y0, y1, y2 = [float(data.array[int(vrtxs[i]) - 1].y) for i in range(3)]
    z0, z1, z2 = [float(data.array[int(vrtxs[i]) - 1].z) for i in range(3)]

    param = np.array([[3000, 0, 500], [0, 3000, 250], [0, 0, 1]]) #задаем параметры
    #z_add - это фиксированное значение, добавляемое к каждой координате z
    x0, y0 = [(np.dot(param, np.array([[x0], [y0], [z0 + z_add]])) / (z0 + z_add))[i] for i in range(2)]
    x1, y1 = [(np.dot(param, np.array([[x1], [y1], [z1 + z_add]])) / (z1 + z_add))[i] for i in range(2)]
    x2, y2 = [(np.dot(param, np.array([[x2], [y2], [z2 + z_add]])) / (z2 + z_add))[i] for i in range(2)]
    
    lam0 = ((x1 - x2) * (y - y2) - (y1 - y2) * (x - x2)) / ((x1 - x2) * (y0 - y2) - (y1 - y2) * (x0 - x2))
    lam1 = ((x2 - x0) * (y - y0) - (y2 - y0) * (x - x0)) / ((x2 - x0) * (y1 - y0) - (y2 - y0) * (x1 - x0))
    lam2 = ((x0 - x1) * (y - y1) - (y0 - y1) * (x - x1)) / ((x0 - x1) * (y2 - y1) - (y0 - y1) * (x2 - x1))

    return Vert(lam0, lam1, lam2)

def drawing_zbuffer_new(vrtxs, data, matrix, zbuffer, z_add):
   
    cos = -norm_cos(vrtxs, data)
    if cos < 0:
        x0, x1, x2 = [float(data.array[int(vrtxs[i]) - 1].x) for i in range(3)]
        y0, y1, y2 = [float(data.array[int(vrtxs[i]) - 1].y) for i in range(3)]
        z0, z1, z2 = [float(data.array[int(vrtxs[i]) - 1].z) for i in range(3)]

        param = np.array([[3000, 0, 500], [0, 3000, 250], [0, 0, 1]])

        x0, y0 = [(np.dot(param, np.array([[x0], [y0], [z0 + z_add]])) / (z0 + z_add))[i] for i in range(2)]
        x1, y1 = [(np.dot(param, np.array([[x1], [y1], [z1 + z_add]])) / (z1 + z_add))[i] for i in range(2)]
        x2, y2 = [(np.dot(param, np.array([[x2], [y2], [z2 + z_add]])) / (z2 + z_add))[i] for i in range(2)]
    
        xmin, xmax = min(x0, x1, x2), max(x0, x1, x2)
        ymin, ymax = min(y0, y1, y2), max(y0, y1, y2)
        zmin, zmax = min(z0, z1, z2), max(z0, z1, z2)

        x_coords = np.arange(int(xmin), math.ceil(xmax))
        y_coords = np.arange(int(ymin), math.ceil(ymax))
        xx, yy = np.meshgrid(x_coords, y_coords, indexing='xy')
        for i in range(len(xx)):
            for j in range(len(xx[i])):
                coord = new_barycentric(xx[i][j], yy[i][j], vrtxs, data, z_add)
                if (coord.x >= 0) & (coord.y >= 0) & (coord.z >= 0):
                    zcoord = coord.x * z0 + coord.y * z1 + coord.z * z2
                    if zcoord < zbuffer[xx[i][j]][yy[i][j]]:
                        matrix[-yy[i][j]][xx[i][j]] = (-250 * cos, -100 * cos, -100 * cos)
                        zbuffer[xx[i][j]][yy[i][j]] = zcoord



zz = np.full((1000, 1000, 3), (0, 0, 0), dtype=np.uint8)
zbufferr = np.inf * np.ones((1000, 1000), dtype=float)

for vrtxs in polygons:
    drawing_zbuffer_new(vrtxs, data, zz, zbufferr, 500)
    newzbuff = Image.fromarray(zz, mode='RGB')
    newzbuff.save('zbuffer_update.jpg')

In [36]:
#16 Поворот модели

aplha, beta, gamma = 30, 180, 50   #углы поворота вокруг осей
alpharad = aplha * math.pi / 180 #запишем в радианах
betarad = beta * math.pi / 180 
gammarad = gamma * math.pi / 180
#матрицы поворота
M1 = np.array([[1, 0, 0],
              [0, math.cos(alpharad), math.sin(alpharad)],
              [0, -math.sin(alpharad), math.cos(alpharad)]])
M2 = np.array([[math.cos(betarad), 0, math.sin(betarad)],
               [0, 1, 0],
               [-math.sin(betarad), 0, math.cos(betarad)]])
M3 = np.array([[math.cos(gammarad), math.sin(gammarad), 0],
               [-math.sin(gammarad), math.cos(gammarad), 0],
               [0, 0, 1]])
R = M1@M2@M3 #вычисляем по формуле
data2 = Read('fox.obj') 
#применение поворота к каждой вершине модели
for i in range(len(data2.array)):
    R_upd = R@np.array([data2.array[i].x, data2.array[i].y, data2.array[i].z]).T
    data2.array[i].x, data2.array[i].y, data2.array[i].z = R_upd[0], R_upd[1], R_upd[2] #обновляем коорд-ты

zbf = np.full((1000, 1000, 3), (0, 0, 0), dtype=np.uint8)
zb = np.inf * np.ones((1000, 1000), dtype=float)

for vrtxs in polygons:
    drawing_zbuffer_new(vrtxs, data2, zbf, zb, 500)
    reverse = Image.fromarray(zbf, mode='RGB')
    reverse.save('reverse.jpg')